Bu projede, bir oyun şirketinin satış verilerini kullanarak kullanıcıları demografik özelliklerine göre segmentlere ayırmayı ve her segmentin şirkete ne kadar kazandırabileceğini tahmin etmeyi hedefledim.

Veri setinde yer alan bilgiler:
- Kullanıcının ülkesi
- Cihaz türü (Android/iOS)
- Cinsiyeti
- Yaşı
- Ve yaptığı harcama (PRICE)

Adım adım şu analizleri yaptım:
1. Veriyi tanıdım ve temel istatistikleri inceledim
2. Yaşı aralıklara bölerek kategorik hale getirdim
3. Tüm demografik kırılımları birleştirip “persona” oluşturdum
4. Bu personaları PRICE’a göre segmentlere ayırdım (A, B, C, D)
5. Yeni gelen bir kullanıcıyı segmentine atayarak ortalama gelir tahmin ettim

Bu analiz sayesinde şirket:
- Kimi hedeflemesi gerektiğini
- Hangi müşteri ne kadar değerli
- Kimin için nasıl strateji geliştirilmeli gibi
sorulara kolayca cevap verebilir.

Haydi birlikte keşfedelim!

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/persona/persona.csv


In [2]:
df = pd.read_csv("/kaggle/input/persona/persona.csv")

df.head() # ilk 5 satır

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PRICE,5000.0,34.1320,12.464897,9.0,29.0,39.0,39.0,59.0
AGE,5000.0,23.5814,8.995908,15.0,17.0,21.0,27.0,66.0


## Sayısal Değişkenlerin Genel Özeti

Veri setinde toplamda **5000 gözlem** var. Hem `PRICE` (harcama miktarı) hem de `AGE` (yaş) sütunlarında eksik değer yok gibi görünüyor. İşte dikkatimi çeken bazı noktalar:

### PRICE

- **Ortalama harcama:** 34.13
- **Min - Max aralığı:** 9 TL ile 59 TL arasında değişiyor.
- **En sık karşılaşılan değerler:** Q1=29, Q2 (median)=39, Q3=39 → Yani büyük bir kısmı 29-39 bandında harcama yapmış.
- **Standart sapma:** ~12.46 → Harcamalar çok uçuk değil, genel olarak ortalamaya yakın seyrediyor.

Bu ne demek oluyor ki çok pahalı alışveriş yapanlar yok, genelde harcamalar sınırlı ve oyun içi küçük satın alımları temsil ediyor olabilir.



### AGE 

- **Ortalama yaş:** 23.58
- **Min yaş:** 15, **Max yaş:** 66
- **Q1=17**, **Medyan=21**, **Q3=27** → Kullanıcıların çoğu **17-27 yaş** arasında, yani genç bir kitleye hitap ediliyor.
- **Standart sapma:** 9 → Yaşlar geniş bir dağılıma sahip ama ağırlık gençlerde.

Bu veri, oyun şirketinin hitap ettiği kitlenin çoğunlukla **genç bireyler** olduğunu gösteriyor. Segmentasyon yaparken yaşları kategorilere ayırmak çok mantıklı olacak!



In [5]:
df.isnull().sum()

PRICE      0
SOURCE     0
SEX        0
COUNTRY    0
AGE        0
dtype: int64

In [6]:
def check_outlier(col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return df[(df[col] < lower) | (df[col] > upper)]

# PRICE için
outliers_price = check_outlier("PRICE")
print("PRICE Aykırı Değer Sayısı:", outliers_price.shape[0])

# AGE için
outliers_age = check_outlier("AGE")
print("AGE Aykırı Değer Sayısı:", outliers_age.shape[0])


PRICE Aykırı Değer Sayısı: 412
AGE Aykırı Değer Sayısı: 258


In [7]:
# PRICE için aykırı deger yuzdesi
price_outlier_ratio = 100 * outliers_price.shape[0] / df.shape[0]
print(f"PRICE Aykırı Değer Yüzdesi: {price_outlier_ratio:.2f}%")

# AGE için aykırı deger yuzdesi
age_outlier_ratio = 100 * outliers_age.shape[0] / df.shape[0]
print(f"AGE Aykırı Değer Yüzdesi: {age_outlier_ratio:.2f}%")


PRICE Aykırı Değer Yüzdesi: 8.24%
AGE Aykırı Değer Yüzdesi: 5.16%


## Aykırı Değer Tespiti

PRICE sütununda aykırı değer oranı **%8.24**, AGE için ise **%5.16** çıktı.

> Bu ne demek?  
Verinin çok büyük bir kısmı normal sınırlar içinde, ama PRICE tarafında bir tık daha fazla uç değer var. Özellikle oyun sektöründe bu tarz "çok harcayan" kullanıcılar (whale dediğimiz grup) normal kabul ediliyor.

O yüzden bu aykırıları temizlemek yerine şimdilik elimde tutuyorum.  
Verinin doğal yapısına uygun olduklarını düşünüyorum.

AGE tarafında da benzer durum var. Çok yaşlı veya çok genç kullanıcılar olabilir ama oran çok yüksek değil. İleride gerekirse baskılama (winsorizing) ya da dönüşüm yapılabilir ama şu an için gerek görmedim.



## Grouping by Demographic Features to Calculate Average Revenue
Bu adımda kullanıcıları ülke, cihaz türü, cinsiyet ve yaş bazında gruplayarak her bir grubun ortalama harcama tutarını hesapladım.  
Bu veriler ileride seviye bazlı persona oluşturmada ve segment atamada temel olacak.


In [8]:
# q1: kaç gözlem ve kaç değişken 
print(f"Gözlem sayısı: {df.shape[0]}")
print(f"Sütun sayısı: {df.shape[1]}")


Gözlem sayısı: 5000
Sütun sayısı: 5


In [9]:
# q2: kaç tane unique source var ve fiyatları ne
print(df["SOURCE"].value_counts())
print("Unique source sayısı:", df["SOURCE"].nunique())


SOURCE
android    2974
ios        2026
Name: count, dtype: int64
Unique source sayısı: 2


Veri setinde sadece **2 farklı cihaz türü** bulunuyor: Android ve iOS.  
Android kullanıcıları iOS’a göre daha fazla ve bu da analizlerde cihaz bazlı farklara dikkat etmemiz gerektiğini gösteriyor.

In [10]:
# q3: kaç farklı price var
print("Unique PRICE sayısı:", df["PRICE"].nunique())


Unique PRICE sayısı: 6


Toplamda **6 farklı fiyat** değeri var.  
Yani oyuncular ya sabit tutarlı ürünler alıyor ya da şirketin sunduğu paketler belli aralıklarda. Bu sınırlı çeşit, segmentasyon sonrası ortalama değerlerin net ve yönetilebilir olmasını sağlar.


In [11]:
# q4: hangi PRICE'dan kaçar tane satış gerçekleşmiş?
df["PRICE"].value_counts()


PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

En çok 29 TL’lik satış yapılmış. Bu fiyat seviyesinin en popüler olduğunu söyleyebiliriz.Yani bu tutarda sunulan içerikler kullanıcılar için en çekici olanlar gibi görünüyor.  
Bu bilgi ileride “ortalama kullanıcı değeri” tahmininde faydalı olabilir.


In [12]:
# q5: hangi ülkeden ne kadar satış
df["COUNTRY"].value_counts()


COUNTRY
usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: count, dtype: int64

En çok satış **USA** ve **BRA** ülkelerinde gerçekleşmiş.  
Bu iki ülke müşteri kitlesinin büyük kısmını oluşturuyor olabilir.  
Segmentasyon yaparken ülke bazlı strateji oluşturmak gerekebilir.

In [13]:
# q6: ülkelere göre toplam kazanç
df.groupby("COUNTRY")["PRICE"].sum()

COUNTRY
bra    51354
can     7730
deu    15485
fra    10177
tur    15689
usa    70225
Name: PRICE, dtype: int64

En yüksek gelir yine **USA**’dan gelmiş.Sadece kullanıcı sayısı değil, aynı zamanda kullanıcıların harcama eğilimleri de yüksek.

In [14]:
# q7: cihaz türlerine göre satış sayısı
df["SOURCE"].value_counts()

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

Veri setinde toplam **2974 Android** ve **2026 iOS** kullanıcısı bulunuyor.  
Yani kullanıcıların yaklaşık **%60'ı Android**, **%40'ı iOS** cihaz kullanıyor.

Bu fark demek oluyor ki Android kullanıcıları çoğunlukta, ancak ileride gelir analizinde göreceğiz ki **iOS kullanıcıları daha yüksek harcama yapabiliyor** olabilir.  
O yüzden sadece sayı değil, kalite (gelir potansiyeli) de önemlidir. Cihaz bazlı segmentler oluştururken bu dağılımı göz önünde bulunduracağım.

In [15]:
# q8: ülkelere göre ortalama kazanç
df.groupby("COUNTRY")["PRICE"].mean().round(2)

COUNTRY
bra    34.33
can    33.61
deu    34.03
fra    33.59
tur    34.79
usa    34.01
Name: PRICE, dtype: float64

Her bir ülkenin kullanıcı başına ortalama gelirine baktığımda değerler birbirine oldukça yakın olsa da bazı küçük farklar dikkat çekiyor:

- **Türkiye (tur)** en yüksek ortalama harcamaya sahip: **34.79 TL**  
- **Brezilya (bra)** ve **USA** da 34 TL civarında; bu da bu ülkelerde kullanıcıların istikrarlı harcama yaptığını gösteriyor.  
- **Kanada (can)** ise listenin en altında, ortalama **33.61 TL** ile.

Aradaki farklar küçük gibi görünse de büyük veri setlerinde bu farklar toplam gelire ciddi etki edebilir. Segmentasyon aşamasında bu farklar üzerinden ülke bazlı persona değerlendirmeleri yapılabilir.

In [16]:
# q9: SOURCE'a göre ortalama kazanç
df.groupby("SOURCE")["PRICE"].mean().round(2)

SOURCE
android    34.17
ios        34.07
Name: PRICE, dtype: float64

Android kullanıcıları ortalama **34.17 TL**, iOS kullanıcıları ise **34.07 TL** harcamış.  
Yani aralarında neredeyse hiç fark yok gibi ama yine de **Android bir tık önde**.

Bu tabloyu görünce açıkçası “iOS daha çok harcıyor olabilir” diye düşünüyordum ama öyle çıkmadı. Demek ki bu projede cihazdan çok başka şeyler (ülke, yaş grubu vs.) daha belirleyici olacak.

Yine de kullanıcı sayısı Android’de fazla olduğu için toplam kazançta onlar daha baskın olabilir.  

> Ama unutmayalım: **fazla kişi ≠ fazla para her zaman değil.**




In [17]:
# q10: COUNTRY-SOURCE kırılımında ortalama kazanç
df.groupby(["COUNTRY","SOURCE"])["PRICE"].mean().round(2)

COUNTRY  SOURCE 
bra      android    34.39
         ios        34.22
can      android    33.33
         ios        33.95
deu      android    33.87
         ios        34.27
fra      android    34.31
         ios        32.78
tur      android    36.23
         ios        33.27
usa      android    33.76
         ios        34.37
Name: PRICE, dtype: float64

Bazı ülkelerde Android önde, bazı ülkelerde iOS ama farklar baya ilginç.

- Mesela **Türkiye’de Android kullanıcıları** uçmuş resmen! Ortalama 36.23 TL harcamışlar, iOS kullanıcıları baya geride kalmış.
- **USA'da tam tersi**, iOS kullanıcıları Android'e göre daha fazla harcıyor. Bu çok şaşırtmadı aslında.
- **Fransa'da iOS kullanıcıları baya düşük kalmış**.

Genel olarak:
- Her ülkenin kendi içinde farklı bir kullanıcı profili var.
- Sadece ülkeye ya da sadece cihaza bakmak yetmezmiş, **ikisi birlikte analiz edilince gerçek tablo ortaya çıkıyor.**

Bu detaylar ileride segment oluştururken resmen altın değerinde.

## COUNTRY, SOURCE, SEX, AGE Kırılımında Ortalama   Kazançlar
Amaç: Her kullanıcının ülke + cihaz + cinsiyet + yaş kombinasyonuna göre ortalama ne kadar gelir getirdiğini hesaplama

In [20]:
agg_df = df.groupby(["COUNTRY", "SOURCE", "SEX", "AGE"]).agg({"PRICE": "mean"}).sort_values("PRICE", ascending=False).reset_index()
agg_df


,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0
...,...,...,...,...,...
343,usa,ios,female,38,19.0
344,usa,ios,female,30,19.0
345,can,android,female,27,19.0
346,fra,android,male,18,19.0


Burada artık işi biraz daha derinleştirdim.  
Kullanıcıları **ülke + cihaz + cinsiyet + yaş** kombinasyonlarına göre gruplayıp ortalama gelirlerini hesapladım. Toplamda 348 farklı profil ortaya çıktı.

İlk satırlarda gördüğüm kombinasyonlar (örneğin: `BRA-ANDROID-MALE-46`, `USA-ANDROID-MALE-36`) yüksek gelir getiren gruplar.  
**Bu tür kullanıcılar benim için yüksek potansiyel segmentler.**

Tablonun altlarına indiğimde ise 9–19 TL bandında kalan kullanıcılar var.  
Bu da bana gösteriyor ki her profil eşit gelir getirmiyor; segmentasyon bu yüzden kritik.

Bu çıktıyı ileride yaş kategorileri ile daha okunabilir hale getireceğim ve “persona” haline dönüştüreceğim.  
Yani artık her kullanıcıyı temsil eden seviye tabanlı bir kimlik tanımı yapabileceğim.


## `AGE` Değişkenini Kategorik Hale Getirme

In [23]:
bins = [0, 18, 23, 30, 40, 66]  # yaş aralıkları
labels = ["0_18", "19_23", "24_30", "31_40", "41_66"]

agg_df["AGE_CAT"] = pd.cut(agg_df["AGE"], bins=bins, labels=labels)
agg_df


,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,bra,android,male,46,59.0,41_66
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40
...,...,...,...,...,...,...
343,usa,ios,female,38,19.0,31_40
344,usa,ios,female,30,19.0,24_30
345,can,android,female,27,19.0,24_30
346,fra,android,male,18,19.0,0_18


Bu adımda yaşları daha anlamlı ve gruplandırılmış bir şekilde kullanmak için `AGE` sütununu kategorik hale getirdim.  
Yeni oluşturduğum `AGE_CAT` sütunuyla artık kullanıcıları yaş aralıklarına göre sınıflandırabiliyorum.

Örneğin:  
- 46 yaşındaki biri → `41_66` grubunda  
- 24 yaşındaki biri → `24_30` grubunda  
- 18 yaşındaki biri → `0_18` grubunda yer aldı.

Bu gruplar sayesinde kullanıcıları analiz etmek, segmentlere ayırmak ve gelecekte persona oluşturmak çok daha kolay hale geldi.  
Ayrıca bu sayede yaş bazlı davranış farklılıklarını da daha net gözlemleyebileceğim.

## Seviye tabanlı müşterileri (persona) 

In [24]:
agg_df["customers_level_based"] = (
    agg_df["COUNTRY"].str.upper() + "_" +
    agg_df["SOURCE"].str.upper() + "_" +
    agg_df["SEX"].str.upper() + "_" +
    agg_df["AGE_CAT"].astype(str)
)


In [25]:
persona_df = agg_df.groupby("customers_level_based").agg({"PRICE": "mean"}).reset_index()


In [26]:
persona_df.head()


,customers_level_based,PRICE
0,BRA_ANDROID_FEMALE_0_18,35.645303
1,BRA_ANDROID_FEMALE_19_23,34.077340
2,BRA_ANDROID_FEMALE_24_30,33.863946
3,BRA_ANDROID_FEMALE_31_40,34.898326
4,BRA_ANDROID_FEMALE_41_66,36.737179


## Yeni müşterileri (personaları) segmentlere ayırma

Yapılacaklar:
- Her bir persona’yı PRICE değerine göre 4 gruba ayrılacak: A, B, C, D

- Bu grupları SEGMENT adında yeni bir sütun olarak eklenecek

- Segmentleri istatistiksel olarak mean, max, sum gibi metriklerle incelenecek

In [29]:
persona_df["SEGMENT"] = pd.qcut(persona_df["PRICE"], 4, labels=["D", "C", "B", "A"])


In [30]:
persona_df.groupby("SEGMENT").agg({
    "PRICE": ["mean", "max", "sum", "count"]
})


/tmp/ipykernel_35/1401674770.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  persona_df.groupby("SEGMENT").agg({


PRICE                              
              mean        max          sum count
SEGMENT                                         
D        29.206780  32.333333   817.789833    28
C        33.509674  34.077340   904.761209    27
B        34.999645  36.000000   944.990411    27
A        38.691234  45.428571  1044.663328    27

Her personaları PRICE değerine göre 4 segmente ayırdım: A, B, C ve D.  
Ve açıkçası sonuçlar tam beklediğim gibi geldi.

- **A Segmenti**: En değerli müşteri grubu. Ortalama harcama neredeyse 39 TL. Toplam kazançları da diğerlerinden açık ara fazla.  
  Buradaki kullanıcılar VIP olarak ele alınabilir.  
- **B Segmenti**: Orta-üst seviye kullanıcılar. Ortalama 35 TL harcıyorlar, gayet stabil bir grup.  
- **C Segmenti**: Bir tık daha düşük ama hâlâ fena değil, ortalama 33.5 TL.  
- **D Segmenti**: En düşük ortalamaya sahip grup ama bu grupta **28 kişi** var. Sayıca fazla olmaları, toplamda hâlâ önemli bir katkı sunduklarını gösteriyor.

Yani sadece ortalamaya değil, toplam harcamaya da bakmak gerek.


##  Yeni gelen müşterileri sınıflandırıp, ne kadar gelir getirebileceklerini  tahmin etme

In [31]:
user1 = "TUR_ANDROID_FEMALE_31_40"
user2 = "FRA_IOS_FEMALE_31_40"


In [33]:
persona_df[persona_df["customers_level_based"] == user1]



,customers_level_based,PRICE,SEGMENT
72,TUR_ANDROID_FEMALE_31_40,41.833333,A


Elimde 33 yaşında, Android kullanan bir Türk kadını profili vardı.  
Bu bilgileri kullanarak `TUR_ANDROID_FEMALE_31_40` persona’sına karşılık geldiğini belirledim.

Çıkan sonuç:

- **Segment:** A  
- **Ortalama Gelir:** 41.83 TL

Yani bu profil, en yüksek kazanç potansiyeline sahip kullanıcı segmentine ait.  
A segmentinde yer alması, bu kullanıcı grubunun şirkete değerli bir katkı sunduğunu gösteriyor.

Böyle bir kullanıcıyı kazanmak ve elde tutmak için özel teklifler ya da hedefli kampanyalar oluşturmak mantıklı olur. 

In [34]:
persona_df[persona_df["customers_level_based"] == user2]

,customers_level_based,PRICE,SEGMENT
63,FRA_IOS_FEMALE_31_40,32.818182,C


Bu sefer elimde 35 yaşında, iOS kullanan bir Fransız kadını vardı.  
Profilini `FRA_IOS_FEMALE_31_40` şeklinde oluşturdum ve çıktılara baktım.

Sonuç:

- **Segment:** C  
- **Ortalama Gelir:** 32.82 TL

Bu müşteri profili C segmentinde yer alıyor. Yani aşırı düşük değil ama A segmentine göre daha düşük harcama potansiyeline sahip bir grup.

Buna rağmen bu kullanıcıların da değeri var çünkü iOS tarafında ve orta seviye harcama yapıyorlar.  
Doğru kampanya ile bu profilin A segmentine çekilme ihtimali var.

## Proje Özeti ve Kapanış

Bu projede veriye dayalı olarak müşteri segmentleri oluşturdum ve her bir profilin şirkete sağlayabileceği ortalama geliri hesapladım.

Neler başardım?
- Kullanıcıları anlamlı aralıklara bölüp kategorize ettim  
- Seviye tabanlı müşteri tanımı (persona) oluşturdum  
- Her bir persona’yı PRICE’a göre segmentledim  
- Yeni gelen müşterileri sınıflandırarak değer tahmini yaptım

En çok dikkatimi çeken kısımlar:
- Bazı ülkelerde cihaz tercihinin gelir üzerindeki etkisi
- Aynı yaş grubunun farklı ülkelerde çok farklı segmentlerde yer alması
- Her profilin stratejik öneminin değişmesi

Bu analiz pazarlama stratejisi belirlemekten hedefli kampanyalara kadar birçok noktada şirketlere yol gösterebilir.

## Gelecekte Neler Yapılabilir?

Bu proje temel bir segmentasyon ve gelir tahmini yapısını başarılı şekilde kurdu. Ancak ileride şu geliştirmeler yapılabilir:

- **Zaman Serisi Takibi:** Kullanıcıların alışveriş geçmişi varsa, zaman içindeki davranışları incelenerek daha dinamik segmentler oluşturulabilir.
- **Makine Öğrenmesi Uygulamaları:** Segmentleri sadece PRICE ile değil, sınıflandırma algoritmalarıyla (Random Forest, XGBoost vb.) tahmin etmek mümkün olabilir.
- **Kullanıcı Davranışlarıyla Zenginleştirme:** Kullanıcının oyun içi davranışları, etkileşim süresi gibi bilgiler eklenirse segmentasyon çok daha anlamlı hale gelir.
- **Kampanya Etki Analizi:** Her segmente farklı kampanya uygulanıp, hangisinin daha fazla gelir getirdiği test edilebilir (A/B test gibi).
- **Uluslararası Pazarlama Stratejileri:** Ülke bazlı davranışlar detaylandırılarak coğrafi hedefleme çalışmaları yapılabilir.
- **Otomatik Segment Güncelleme:** Yeni gelen kullanıcı verisiyle modelin kendini otomatik güncellemesi sağlanabilir.

Bu projenin çıktıları; pazarlama, ürün geliştirme ve kullanıcı kazanımı gibi birçok departman için stratejik karar alma sürecinde kullanılabilir.